# Transformation with transform()

- 針對 GroupBy Object物件 使用的函式

In [1]:
import pandas as pd

In [2]:
titanic = pd.read_csv("../data/titanic_ver01.csv")

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin
0,1,0,3,male,22.0,1,0,7.2500,S,NaN
1,2,1,1,female,38.0,1,0,71.2833,C,C
2,3,1,3,female,26.0,0,0,7.9250,S,NaN
3,4,1,1,female,35.0,1,0,53.1000,S,C
4,5,0,3,male,35.0,0,0,8.0500,S,NaN


In [4]:
titanic.groupby(["Sex", "Pclass"]).Survived.mean()
# 計算出的就是Survived Rate

Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64

---
現在我們想要在原始資料集中，加上一欄式我們剛剛分組計算的結果。  
比如第一筆 [Sex, Pclass] = [male, 3]，對應到的Survived rate是 0.135447  
要做到這件事，我們先生出每一筆對應下的值，變成一個 series 

---
**note04_第二節_講座六.ipynb**
- 這樣取出來的東西稱為pandas series
- series 是指 one dimensionlabeled array，label就是有index

In [5]:
titanic.groupby(["Sex", "Pclass"]).Survived.transform("mean")
# transform中要放我們要轉換的 function，記得用雙引號括起來，像 agg() 的函式用法一樣
# 注意這是 serires，他有index

0      0.135447
1      0.968085
2      0.500000
3      0.968085
4      0.135447
5      0.135447
6      0.368852
7      0.135447
8      0.500000
9      0.921053
10     0.500000
11     0.968085
12     0.135447
13     0.135447
14     0.500000
15     0.921053
16     0.135447
17     0.157407
18     0.500000
19     0.500000
20     0.157407
21     0.157407
22     0.500000
23     0.368852
24     0.500000
25     0.500000
26     0.135447
27     0.368852
28     0.500000
29     0.135447
         ...   
861    0.157407
862    0.968085
863    0.500000
864    0.157407
865    0.921053
866    0.921053
867    0.368852
868    0.135447
869    0.135447
870    0.135447
871    0.968085
872    0.368852
873    0.135447
874    0.921053
875    0.500000
876    0.135447
877    0.135447
878    0.135447
879    0.968085
880    0.921053
881    0.135447
882    0.500000
883    0.157407
884    0.135447
885    0.500000
886    0.157407
887    0.968085
888    0.500000
889    0.368852
890    0.135447
Name: Survived, Length: 

如此一來就可以得到每一筆資料，在分組對應下的 Survived Rate

In [6]:
# 新增的一欄將叫做 group_surv_rate 
titanic["group_surv_rate"] = titanic.groupby(["Sex", "Pclass"]).Survived.transform("mean")

In [7]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin,group_surv_rate
0,1,0,3,male,22.0,1,0,7.2500,S,NaN,0.135447
1,2,1,1,female,38.0,1,0,71.2833,C,C,0.968085
2,3,1,3,female,26.0,0,0,7.9250,S,NaN,0.500000
3,4,1,1,female,35.0,1,0,53.1000,S,C,0.968085
4,5,0,3,male,35.0,0,0,8.0500,S,NaN,0.135447


---
現在要創造一欄叫做Outliers，企圖找出存活下來，那些比較特殊身分的人。  

$$ Outliers = | titanic.Survived - titanic.group\_surv\_rate |$$  


也就是說我要抓出 [Sex, Pclass] = [male, 3]，明明存活機率很低，但是卻存活下來 Survived = 1  
這樣相減， Outliers 值就很大，因為理想上他不會活下來，但是他卻活下來了  
所以會是新聞報導中的Outliers

In [8]:
titanic["Outliers"] = abs(titanic.Survived - titanic.group_surv_rate)

In [9]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin,group_surv_rate,Outliers
0,1,0,3,male,22.0,1,0,7.2500,S,NaN,0.135447,0.135447
1,2,1,1,female,38.0,1,0,71.2833,C,C,0.968085,0.031915
2,3,1,3,female,26.0,0,0,7.9250,S,NaN,0.500000,0.500000
3,4,1,1,female,35.0,1,0,53.1000,S,C,0.968085,0.031915
4,5,0,3,male,35.0,0,0,8.0500,S,NaN,0.135447,0.135447


接著我們就抓出那些很特別的人，定義 Outliers > 0.85

In [10]:
titanic[titanic.Outliers > 0.85]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin,group_surv_rate,Outliers
36,37,1,3,male,NaN,0,0,7.2292,C,NaN,0.135447,0.864553
41,42,0,2,female,27.00,1,0,21.0000,S,NaN,0.921053,0.921053
65,66,1,3,male,NaN,1,1,15.2458,C,NaN,0.135447,0.864553
74,75,1,3,male,32.00,0,0,56.4958,S,NaN,0.135447,0.864553
81,82,1,3,male,29.00,0,0,9.5000,S,NaN,0.135447,0.864553
107,108,1,3,male,NaN,0,0,7.7750,S,NaN,0.135447,0.864553
125,126,1,3,male,12.00,1,0,11.2417,C,NaN,0.135447,0.864553
127,128,1,3,male,24.00,0,0,7.1417,S,NaN,0.135447,0.864553
146,147,1,3,male,27.00,0,0,7.7958,S,NaN,0.135447,0.864553
165,166,1,3,male,9.00,0,2,20.5250,S,NaN,0.135447,0.864553


In [11]:
titanic[titanic.Outliers > 0.85].info()
# titanic[titanic.Outliers > 0.85].count()
# 可以看出有56筆

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 36 to 869
Data columns (total 12 columns):
PassengerId        56 non-null int64
Survived           56 non-null int64
Pclass             56 non-null int64
Sex                56 non-null object
Age                47 non-null float64
SibSp              56 non-null int64
Parch              56 non-null int64
Fare               56 non-null float64
Embarked           56 non-null object
Cabin              6 non-null object
group_surv_rate    56 non-null float64
Outliers           56 non-null float64
dtypes: float64(4), int64(5), object(3)
memory usage: 5.7+ KB


---
# 課程結束

---
上一個筆記 **note10_第三節_講座十二.ipynb**  
想看看如果分組依照 Sex, Pclass, ad_chi 去分組  
然後用一樣的方法去抓出 Outliers ，看看會不會更少~

In [12]:
titanic["ad_chi"] = "adult"
# 新增一欄ad_chi，先填滿 adult 

In [13]:
titanic.loc[titanic.Age < 18, "ad_chi"] = "child"

In [14]:
# 新增的一欄將叫做 group_surv_rate 
titanic["group_surv_rate2"] = titanic.groupby(["Sex", "Pclass", "ad_chi"]).Survived.transform("mean")

In [15]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin,group_surv_rate,Outliers,ad_chi,group_surv_rate2
0,1,0,3,male,22.0,1,0,7.2500,S,NaN,0.135447,0.135447,adult,0.121711
1,2,1,1,female,38.0,1,0,71.2833,C,C,0.968085,0.031915,adult,0.976744
2,3,1,3,female,26.0,0,0,7.9250,S,NaN,0.500000,0.500000,adult,0.486239
3,4,1,1,female,35.0,1,0,53.1000,S,C,0.968085,0.031915,adult,0.976744
4,5,0,3,male,35.0,0,0,8.0500,S,NaN,0.135447,0.135447,adult,0.121711


In [16]:
titanic["Outliers2"] = abs(titanic.Survived - titanic.group_surv_rate2)

In [17]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin,group_surv_rate,Outliers,ad_chi,group_surv_rate2,Outliers2
0,1,0,3,male,22.0,1,0,7.2500,S,NaN,0.135447,0.135447,adult,0.121711,0.121711
1,2,1,1,female,38.0,1,0,71.2833,C,C,0.968085,0.031915,adult,0.976744,0.023256
2,3,1,3,female,26.0,0,0,7.9250,S,NaN,0.500000,0.500000,adult,0.486239,0.513761
3,4,1,1,female,35.0,1,0,53.1000,S,C,0.968085,0.031915,adult,0.976744,0.023256
4,5,0,3,male,35.0,0,0,8.0500,S,NaN,0.135447,0.135447,adult,0.121711,0.121711


In [18]:
titanic[titanic.Outliers2 > 0.85]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin,group_surv_rate,Outliers,ad_chi,group_surv_rate2,Outliers2
17,18,1,2,male,NaN,0,0,13.0000,S,NaN,0.157407,0.842593,adult,0.082474,0.917526
21,22,1,2,male,34.0,0,0,13.0000,S,D,0.157407,0.842593,adult,0.082474,0.917526
36,37,1,3,male,NaN,0,0,7.2292,C,NaN,0.135447,0.864553,adult,0.121711,0.878289
41,42,0,2,female,27.0,1,0,21.0000,S,NaN,0.921053,0.921053,adult,0.906250,0.906250
65,66,1,3,male,NaN,1,1,15.2458,C,NaN,0.135447,0.864553,adult,0.121711,0.878289
74,75,1,3,male,32.0,0,0,56.4958,S,NaN,0.135447,0.864553,adult,0.121711,0.878289
81,82,1,3,male,29.0,0,0,9.5000,S,NaN,0.135447,0.864553,adult,0.121711,0.878289
107,108,1,3,male,NaN,0,0,7.7750,S,NaN,0.135447,0.864553,adult,0.121711,0.878289
127,128,1,3,male,24.0,0,0,7.1417,S,NaN,0.135447,0.864553,adult,0.121711,0.878289
146,147,1,3,male,27.0,0,0,7.7958,S,NaN,0.135447,0.864553,adult,0.121711,0.878289


In [19]:
titanic[titanic.Outliers2 > 0.85].info()
# 就變成54筆了!!
# 所以有兩名可能是因為年齡的關係，而沒有存活

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 17 to 854
Data columns (total 15 columns):
PassengerId         54 non-null int64
Survived            54 non-null int64
Pclass              54 non-null int64
Sex                 54 non-null object
Age                 43 non-null float64
SibSp               54 non-null int64
Parch               54 non-null int64
Fare                54 non-null float64
Embarked            54 non-null object
Cabin               6 non-null object
group_surv_rate     54 non-null float64
Outliers            54 non-null float64
ad_chi              54 non-null object
group_surv_rate2    54 non-null float64
Outliers2           54 non-null float64
dtypes: float64(6), int64(5), object(4)
memory usage: 6.8+ KB


In [20]:
titanic[titanic.Outliers > 0.85].index

Int64Index([ 36,  41,  65,  74,  81, 107, 125, 127, 146, 165, 177, 199, 204,
            207, 220, 261, 267, 271, 283, 286, 297, 301, 312, 338, 348, 357,
            391, 400, 414, 429, 444, 455, 489, 498, 509, 510, 553, 569, 579,
            622, 643, 664, 692, 709, 744, 751, 762, 772, 788, 803, 804, 821,
            828, 838, 854, 869],
           dtype='int64')

In [21]:
titanic[titanic.Outliers2 > 0.85].index

Int64Index([ 17,  21,  36,  41,  65,  74,  81, 107, 127, 146, 177, 199, 204,
            207, 226, 267, 271, 283, 286, 288, 297, 301, 312, 338, 357, 391,
            400, 414, 429, 444, 455, 498, 509, 510, 543, 547, 553, 569, 570,
            579, 622, 643, 664, 673, 692, 709, 744, 762, 772, 804, 821, 828,
            838, 854],
           dtype='int64')

第一個方法是Outliers的index名單  
36,  41,  65,  74,  81, 107, 125, 127, 146, 165, 177, 199, 204, 207, 220, 261, 267, 271, 283, 286, 297, 301, 312, 338, 348, 357, 391, 400, 414, 429, 444, 455, 489, 498, 509, 510, 553, 569, 579, 622, 643, 664, 692, 709, 744, 751, 762, 772, 788, 803, 804, 821, 828, 838, 854, 869  

第二個方法是Outliers的index名單  
17,  21,  36,  41,  65,  74,  81, 107, 127, 146, 177, 199, 204, 207, 226, 267, 271, 283, 286, 288, 297, 301, 312, 338, 357, 391, 400, 414, 429, 444, 455, 498, 509, 510, 543, 547, 553, 569, 570, 579, 622, 643, 664, 673, 692, 709, 744, 762, 772, 804, 821, 828, 838, 854  

---
有在第一個，而沒有在第二個  
125, 165, 220, 261, 348, 489, 751, 788, 803, 869  

有在第二個，而沒有在第一個  
17, 21, 226, 288, 543, 547, 570, 673